In [1]:
import aiohttp

# Add parent dir to path
import os,sys
import io

sys.path.insert(0,os.environ['PWD'])

import netaddr

from rpki_analysis.delegated_stats import read_delegated_stats

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get('https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats') as resp:
        data = read_delegated_stats(io.StringIO(await resp.text()))
        

In [3]:
data.dtypes

rir                        category
country                    category
afi                        category
raw_resource                 object
length                        int64
date            datetime64[ns, UTC]
status                     category
opaque_id                    object
category                   category
resource                     object
dtype: object

In [4]:
resources_by_uuid = data.loc[data.afi != 'asn'].groupby(['opaque_id']) \
  .agg({'resource': lambda grp: netaddr.IPSet(grp)})

In [5]:
resources_by_uuid.resource.apply(lambda x: list(map(str, x.iter_cidrs())))

opaque_id
00004a29a9bbf62af6b8e5b91b760abc                                        [192.149.81.0/24]
000076699efb0f677b8a77052cacd0eb                                        [204.68.168.0/24]
0001f0f8-7af5-4144-8069-8bcb938afb99                                     [83.243.24.0/21]
0004cc00-194f-4e6b-83fa-430c6b1e25cc                                 [2001:67c:1034::/48]
00057f30-ef27-478d-aa11-1e7d2c8bf67c                    [45.153.144.0/22, 2a0f:3d40::/29]
                                                              ...                        
ffff0239b0911043080384096bee384b                                         [208.68.80.0/22]
iana                                    [160.20.241.0/24, 160.20.245.0/24, 192.47.28.0...
ietf                                    [0.0.0.0/8, 10.0.0.0/8, 100.64.0.0/10, 127.0.0...
lacnic                                  [45.6.236.0/22, 45.68.33.0/24, 45.68.56.0/21, ...
ripencc                                 [45.88.224.0/22, 45.145.36.0/22, 46.255.24.0/2...


In [6]:
resources_by_uuid['ipv4_count'] = resources_by_uuid.resource.apply(lambda res: sum([cidr.size if cidr.version == 4 else 0 for cidr in res.iter_cidrs()]))

In [7]:
resources_by_uuid['ipv4_count'].reset_index().sort_values(['ipv4_count'])

,opaque_id,ipv4_count
53551,96d71713b022d9b63365ab30f8154c9c,0
47935,81fae7555e311369b0290c7fe7616bd9,0
91423,bdf1235f-3296-434a-b2e3-6de246ef7ba8,0
36098,57722611-75ba-4b70-a31a-38537523fe15,0
91425,bdf4f5c5-24c9-4340-9045-b1100f4bc712,0
...,...,...
27969,3a7b40ab58f6de4849b12c87e5806b7c,70056448
32400,4a8a91b5b89d3f900098ebf73ca0b118,77101824
101033,e220ed812a4c17f6a76b9b930dae7f50,80084224
90717,bb474b75b6f23182ffa56daf1cf9ec23,221958912
